* 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성
* ```torch.nn``` 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공
* PyTorch의 모든 모듈은 ```nn.Module``` 의 하위 클래스(subclass) 
* 신경망은 다른 모듈(계층; layer)로 구성된 모듈
* 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 1. 학습을 위한 장치 얻기

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## 2. 클래스 정의
신경망 모델을 ```nn.Module```의 하위 클래스로 정의하고, ```__init__```에서 신경망 계층들을 초기화

```nn.Module```을 상속받은 모든 클래스는 ```forward```메소드에 입력 데이터에 대한 연산들을 구현

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# NeuralNetwork 의 인스턴스(instance)를 생성하고 이를 device 로 이동한 뒤, 구조(structure)를 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device) # 모델을 사용하기 위해 입력 데이터 전달
logits = model(X) # 일부 백그라운드 연산들과 함께 모델 forward 실행(직접 호출 X)
# 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 10차원 텐서 반환
pred_probab = nn.Softmax(dim=1)(logits) # 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률 얻기
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1])


## 3. 모델 계층(Layer)
* FashionMNIST 모델의 계층들을 살펴보자.
* 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보자.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환 (dim=0의 미니배치 차원은 유지)

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
선형 계층은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)을 만든다. 비선형 활성화는 선형 변환 후에 적용되어 *비선형성(nonlinearity)*을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1742,  0.1386, -0.2316, -0.0953,  0.4639,  0.3334, -0.0016,  0.0311,
          0.5542,  0.4364,  0.0637,  0.2673, -0.4958, -0.2602, -0.2874, -0.2900,
         -0.2701,  0.3370, -0.0175,  0.5367],
        [-0.0568,  0.1676, -0.3247,  0.0097,  0.1129, -0.0419,  0.3356,  0.0125,
          0.8699,  0.2167,  0.4204,  0.0671, -0.3244, -0.3547, -0.5057, -0.3696,
         -0.1356,  0.5221,  0.2567,  0.1343],
        [-0.0085,  0.2806, -0.0018, -0.0078, -0.0056, -0.0142,  0.3130, -0.2285,
          0.5470,  0.3347, -0.0087,  0.2989, -0.3251,  0.0090, -0.4412, -0.2020,
         -0.0846,  0.2791, -0.2149,  0.2963]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1742, 0.1386, 0.0000, 0.0000, 0.4639, 0.3334, 0.0000, 0.0311, 0.5542,
         0.4364, 0.0637, 0.2673, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3370,
         0.0000, 0.5367],
        [0.0000, 0.1676, 0.0000, 0.0097, 0.1129, 0.0000, 0.3356, 0.0125, 0.8699,
         0.2167, 0.4204, 0.0671, 0.0000, 0.0000, 0.000

### nn.Sequential
* nn.Sequential 은 순서를 갖는 모듈의 컨테이너
* 데이터는 정의된 것과 같은 순서로 모든 모듈을 통해 전달
* 순차 컨테이너(sequential container)를 사용해 아래의 ```seq_modules```와 같은 신경망을 만들 수 있다.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax
* 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환
* logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)
* ```dim``` 매개변수는 값의 합이 1이 되는 차원

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 4. 모델 매개변수
* 신경망 내부의 많은 계층들은 *매개변수화(parameterize)*된다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다.
* ```nn.Module```을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 ```parameters()``` 및 ```named_parameters()``` 메소드로 모든 매개변수에 접근할 수 있다.

In [14]:
# 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0231, -0.0318, -0.0150,  ..., -0.0013,  0.0172, -0.0236],
        [ 0.0002,  0.0316,  0.0155,  ..., -0.0180, -0.0247,  0.0126]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0265, 0.0039], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0411, -0.0395, -0.0343,  ...,  0.0252, -0.0358, -0.0313],
        [ 0.0174, -0.0411, -0.0064,  ...,  0.0338, -0.0030, -0.0394]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_st